In [8]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from fiona.crs import from_epsg

In [9]:
# import hexagons geojson file
gdf_hexagons = gpd.read_file('../hexagon_res_9/hexagons.geojson')

In [10]:
gdf_hexagons.shape

(547, 4)

In [11]:
gdf_hexagons.head()

,index,x,y,geometry
0,8930e1caa2bffff,127.090880,37.493859,"POLYGON ((127.09261 37.49364, 127.09186 37.495..."
1,8930e1c8487ffff,127.043237,37.488433,"POLYGON ((127.04497 37.48822, 127.04422 37.489..."
2,8930e1c8c33ffff,127.105979,37.486245,"POLYGON ((127.10771 37.48603, 127.10696 37.487..."
3,8930e1c818bffff,127.076602,37.479954,"POLYGON ((127.07833 37.47974, 127.07759 37.481..."
4,8930e1c881bffff,127.115170,37.470806,"POLYGON ((127.11690 37.47059, 127.11615 37.472..."


In [12]:
# import residential buildings geojson file
gdf_residentials = gpd.read_file('../gangnamGu-building-residential.geojson')

In [13]:
gdf_residentials.head()

,USABILITY,HEIGHT,PNU,GRND_FLR,UGRND_FLR,DONG_NM,COL_ADM_SE,UFID,BLDG_PNU,BD_MGT_SN,ARCHAREA,TOTALAREA,BC_RAT,VL_RAT,geometry
0,01000,7.90,1168011400106410010,2,1,None,11680,1987207556994435369900000000,None,1168011400106410010001998,97.530,265.02,49.21,83.13,"MULTIPOLYGON (((127.08634 37.49399, 127.08628 ..."
1,02000,13.16,1168010800101570024,4,1,None,11680,2003202386534451790800000000,None,1168010800101570024008663,134.900,652.19,54.62,199.76,"MULTIPOLYGON (((127.02788 37.50883, 127.02781 ..."
2,02000,0.00,1168010500100790000,12,1,6,11680,1981205226284463626200000000,None,1168010500100790000014655,781.490,9526.81,0.00,0.00,"MULTIPOLYGON (((127.06025 37.51966, 127.06031 ..."
3,01000,9.80,1168010400100110005,3,0,´Ù°¡±¸¿ë´Üµ¶ÁÖÅÃ(9°¡±¸),11680,1995204058644466407200000000,None,1168010400100110005018473,149.717,393.17,59.76,156.95,"MULTIPOLYGON (((127.04673 37.52192, 127.04665 ..."
4,01000,0.00,1168010500100700004,2,1,None,11680,1978204624064458510800000000,None,1168010500100700004015465,75.370,163.18,0.00,0.00,"MULTIPOLYGON (((127.05317 37.51489, 127.05315 ..."


In [14]:
def count_buildings(polygon):
    '''
    This function is for making a list of buildings that is in a hexagon area.
    -----
    input:
    polygon (Shapely.geometry.Polygon): a hexagon polygon 
    -----
    output:
    precise_matches_list (list): a list of residential buildings
    
    '''
    try:
        buildings_sindex = gdf_residentials.sindex
        possible_matches_index = list(buildings_sindex.intersection(polygon.bounds))
        possible_matches = gdf_residentials.iloc[possible_matches_index]
        # extract the list of buildings that is in a hexagon
        precise_matches_list = possible_matches[possible_matches.intersects(polygon)]['UFID'].tolist()
    except:
        # if there is no buildings that matched within a hexagon, return empty list
        precise_matches_list = []
    return precise_matches_list

In [15]:
# get list of residential buildings in each hexagon
gdf_hexagons['residentials'] = gdf_hexagons['geometry'].apply(lambda x: count_buildings(x))

In [16]:
# count the number of residential buildings in each hexagon
gdf_hexagons['count_residentials'] = gdf_hexagons['residentials'].apply(lambda x: len(x))

In [17]:
# delete the hexagon which doesn't contains any residential buildings
gdf_hexagons = gdf_hexagons.loc[gdf_hexagons['count_residentials']>0]

In [18]:
gdf_hexagons = gdf_hexagons.drop(['residentials', 'count_residentials'], axis=1)

In [19]:
gdf_hexagons.to_file('../hexagons_filtered.geojson', encoding='utf8', driver='GeoJSON')